# RAG Summary Demo

This notebook combines concepts from the previous notebooks into a clean, end-to-end RAG workflow.

## Pipeline Steps
1. Load and chunk a document
2. Embed and store in FAISS
3. Accept user question
4. Retrieve relevant chunks
5. Generate answer using OpenAI

In [ ]:
# Install dependencies
%pip install faiss-cpu sentence-transformers openai -q

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai
import os

# Load document
text = """The Amazon rainforest is the largest tropical rainforest in the world. 
It plays a critical role in regulating the Earth's climate and is home to a diverse range of species. 
Deforestation and climate change threaten this vital ecosystem. 
Efforts to preserve the rainforest include sustainable logging, conservation programs, and international agreements."""

# Chunk
chunks = [s.strip() for s in text.split('.') if len(s) > 20]

# Embed
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

# Index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings).astype("float32"))

In [ ]:
# User query
query = "Why is the Amazon rainforest important?"
query_vec = model.encode([query])
D, I = index.search(np.array(query_vec).astype("float32"), k=2)

retrieved_chunks = [chunks[i] for i in I[0]]
print("Retrieved Chunks:")
print("\n".join(retrieved_chunks))

In [ ]:
# Generate answer using OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

context = "\n".join(retrieved_chunks)
prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=100,
    temperature=0.3
)

print("Answer:", response['choices'][0]['text'].strip())

## Recap
- This notebook demonstrates a complete RAG pipeline.
- It’s ready for adapting to your own documents and questions.